In [2]:
%pip install langchain langchain-community langchain-openai


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import yaml

# https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml
with open("data/open-meteo.yml") as f:
    raw_meteo_api_spec = yaml.load(f, Loader=yaml.Loader)

# https://github.com/langchain-ai/langchain/issues/2456
# in reduce_openapi_spec
#    servers=spec["servers"],
#            ~~~~^^^^^^^^^^^
raw_meteo_api_spec["servers"] = [{"url": "https://api.open-meteo.com"}]
# raw_meteo_api_spec.servers[0]["url"] = "https://api.foo.com"
# print(raw_meteo_api_spec)

from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec
meteo_api_spec = reduce_openapi_spec(raw_meteo_api_spec)


In [8]:
from _lessonshelper.pretty_print_callback_handler import PrettyPrintCallbackHandler
pretty_callback = PrettyPrintCallbackHandler()

from langchain_openai import OpenAI
#agent_llm = OpenAI(temperature=0, callbacks=[pretty_callback])
agent_llm = OpenAI(temperature=0)

In [9]:
from langchain.requests import RequestsWrapper
from langchain_community.agent_toolkits.openapi import planner

wrapper = RequestsWrapper()
meteo_agent = planner.create_openapi_agent(
    meteo_api_spec, requests_wrapper=wrapper, llm=agent_llm
)

user_query = "What are the current weather conditions in Ghent expressed in Celsius?"
answer = meteo_agent.run(user_query)
print(answer)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to get the current weather conditions in Ghent in Celsius
Observation:  1. GET /v1/forecast with query params for Ghent's latitude and longitude
2. Convert temperature from Kelvin to Celsius
Thought: I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /v1/forecast with query params for Ghent's latitude and longitude
2. Convert temperature from Kelvin to Celsius

> Entering new AgentExecutor chain...
1. I need to get the forecast for Ghent's latitude and longitude. I will use the requests_get tool to GET the content from the API.
Action: requests_get
Action Input: {"url": "https://api.open-meteo.com/v1/forecast", "params": {"latitude": 51.05, "longitude": 3.7167}, "output_instructions": "I need to extract the temperature from the response."}
Observation: Error: No temperature data available.
Thought:1. I need to get the forecast for Ghent's latitude an